In [2]:
!pip install spacy transformers nltk
!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [12]:
# Импортируем необходимые библиотеки
import nltk
import spacy
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Устанавливаем зависимости
!pip install nltk spacy transformers
!python -m spacy download en_core_web_sm

# Загрузка моделей
nltk.download('punkt')
nlp = spacy.load('en_core_web_sm')

# Пример текста
text = """
There are five types of schools in the US educational system. They are: kindergarten, elementary school, middle school, high school and private school. Children go to kindergarten when they are 5 years old They go to elementary school from ages 6 through 11 (1-5 grades), middle school from ages 12 through 14 (6-8 grades) and high school from ages 15 through 19 (9-12 grades).

About 90 percent of all children attend public school, which is free. The other 10 percent go I private schools, which often include religious education. They are similar to the public schools but parents must pay for their children to go to these schools. About half of all private schools are run by Catholics.

In the United States, education is mainly the responsibility of state and local governments, not the national government. The amount of money spent on education differs from state to state. The subjects studied also differ a little. The school year usually runs from September to June. At the high school level, there are some specialized schools. They include schools that emphasize vocational subjects like business or auto mechanics. Most high schools are general schools. High school students are often involved in the non-academic activities that their school offers -for example, in drama clubs, sports teams, or the school newspaper.
"""

# Разбор текста
doc = nlp(text)

# Извлечение именованных сущностей
entities = [(ent.text, ent.label_) for ent in doc.ents]

# Извлечение предложений
sentences = [sent.text for sent in doc.sents]

# Пример вывода
print("Entities:", entities)
print("Sentences:", sentences)

# Определяем шаблоны для генерации вопросов
import random

def generate_question(entity, sentence):
    question_templates = {
        "PERSON": ["Who is {}?", "Who was mentioned in the context of {}?"],
        "ORG": ["What organization is {}?", "Which organization is related to {}?"],
        "GPE": ["Where is {}?", "What place is referred to as {}?"],
        "DATE": ["When did {} happen?", "What date is {}?"]
    }

    entity_text, entity_label = entity
    if entity_label in question_templates:
        template = random.choice(question_templates[entity_label])
        return template.format(entity_text)
    return None

# Генерация вопросов на основе шаблонов
questions = []
for entity in entities:
    for sentence in sentences:
        if entity[0] in sentence:
            question = generate_question(entity, sentence)
            if question:
                questions.append(question)

# Пример вывода вопросов
print("Template-based Questions:")
for question in questions:
    print(question)

# Генерация вопросов с использованием модели T5
# Загрузка модели T5
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

def generate_t5_question(context, answer):
    input_text = f"generate question: {context} answer: {answer}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    outputs = model.generate(input_ids)
    question = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return question

# Генерация вопросов с использованием T5
t5_questions = []
for entity in entities:
    for sentence in sentences:
        if entity[0] in sentence:
            question = generate_t5_question(sentence, entity[0])
            t5_questions.append(question)

# Пример вывода вопросов, сгенерированных T5
print("T5-based Questions:")
for question in t5_questions:
    print(question)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Entities: [('five', 'CARDINAL'), ('US', 'GPE'), ('5 years old', 'DATE'), ('11', 'CARDINAL'), ('1', 'CARDINAL'), ('14', 'CARDINAL'), ('6', 'CARDINAL'), ('9-12', 'CARDINAL'), ('About 90 percent', 'PERCENT'), ('10 percent', 'PERCENT'), ('About half', 'CARDINAL'), ('Catholics', 'NORP'), ('the United States', 'GPE'), ('The school year', 'DATE'), ('September to June', 'DATE')]
Sentences: ['\nThere are five types of schools in the US educational system.', 'They are: kindergarten, elementary school, middle school, high school and private school.', 'Children go to kindergarten when they are 5 years old They go to elementary school from ages 6 through 11 (1-5 grades), middle school from ages 12 through 14 (6-8 grades) and high school from ages 15 through 19 (9-12 grades).\n\n', 'About 90 percent of all children attend public school, which is free.', 'The other 10 percent go I private schools, which often include religious education.', 'They are similar to the public schools but parents must pay 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5-based Questions:
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
